### Импорт библиотек

In [123]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest, f_regression


### Загрузка данных

In [124]:
data = pd.read_excel('data/data_ford_price.xlsx') 

data

,price,year,condition,cylinders,odometer,title_status,transmission,drive,size,lat,long,weather
0,43900,2016,4,6,43500,clean,automatic,4wd,full-size,36.471500,-82.483400,59.0
1,15490,2009,2,8,98131,clean,automatic,4wd,full-size,40.468826,-74.281734,52.0
2,2495,2002,2,8,201803,clean,automatic,4wd,full-size,42.477134,-82.949564,45.0
3,1300,2000,1,8,170305,rebuilt,automatic,4wd,full-size,40.764373,-82.349503,49.0
4,13865,2010,3,8,166062,clean,automatic,4wd,NaN,49.210949,-123.114720,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
7012,22500,2015,3,6,23500,clean,automatic,rwd,full-size,32.680700,-117.169800,59.0
7013,5975,2005,2,8,0,clean,automatic,rwd,full-size,38.213303,-85.785762,50.0
7014,9999,2006,3,8,161514,clean,automatic,NaN,full-size,37.609783,-120.995406,59.0
7015,10900,2011,2,8,164000,clean,automatic,4wd,full-size,43.140600,-93.385000,47.0


### Предобработка данных

In [125]:
data.shape

(7017, 12)

In [126]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7017 entries, 0 to 7016
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         7017 non-null   int64  
 1   year          7017 non-null   int64  
 2   condition     7017 non-null   int64  
 3   cylinders     7017 non-null   int64  
 4   odometer      7017 non-null   int64  
 5   title_status  7017 non-null   object 
 6   transmission  7017 non-null   object 
 7   drive         6626 non-null   object 
 8   size          5453 non-null   object 
 9   lat           7017 non-null   float64
 10  long          7017 non-null   float64
 11  weather       6837 non-null   float64
dtypes: float64(3), int64(5), object(4)
memory usage: 658.0+ KB


In [127]:
data.nunique()

price           1250
year              44
condition          6
cylinders          6
odometer        2563
title_status       5
transmission       3
drive              3
size               4
lat             3012
long            3018
weather           25
dtype: int64

В трех признаках присутствуют пропуски: 'drive', 'size' и 'weather' - просто удалим записи с пропусками.

In [128]:
data.dropna(inplace = True)

In [129]:
data.shape

(5202, 12)

Количество записей уменьшилось с 7017 до 5202.

Оставим все числовые признаки (хотя 'condition' и 'cylinders'являются категориальными по сути - всего по 6 уникальных значений, 'year' - признак даты, а 'weather' тоже не является непрерывным числовым признаком).

In [130]:
data = data[['price','year', 'condition', 'cylinders', 'odometer', 'lat', 'long', 'weather']]

Выделим целевую переменную и набор признаков и разделим совокупность данных на тренировочную и тестовую выборки с долей тестовой 30 %.

In [131]:
y = data['price']
x = data.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

### Выделение трех важных признаков. Метод рекурсивного исключения признаков.

Выделим три наиболее важных признака с помощью метода рекурсивного исключения признаков.

In [132]:
# создаем объект класса модель линейной регрессии, который будет оценщиком
estimator = LinearRegression()
# создаем объект класса RFE, передавая ему оценщика - модель линейной регрессии,
# требуемое количество признаков - 3 штуки и удалять будем по одному признаку
selector = RFE(estimator, n_features_to_select=3, step=1)
# применяем к созданному объекту RFE метод fit передавая тренировочные выборку с таргетом 
selector = selector.fit(X_train, y_train)
# выводим полученные три важнейших признака 
selector.get_feature_names_out()

array(['year', 'condition', 'cylinders'], dtype=object)

Создадим выборки только с этими тремя важными признаками.

In [133]:
X_train_RFE = X_train.loc[:, ['year', 'condition', 'cylinders']]
X_test_RFE = X_test.loc[:, ['year', 'condition', 'cylinders']]

Создадим и обучим модель линейной регрессии на выбранных трех важных признаках. Сделаем предсказание на тестовой выборке.

In [134]:
# создаем обект класса модель линейной регрессии
model = LinearRegression()
# обучаем модель на тренировочных выборках
model.fit(X_train_RFE, y_train)
# делаем предсказание на тестовой выборке
y_predicted = model.predict(X_test_RFE)

Оценим резудьтат предсказания с помощью метрики MAE.

In [135]:
mae = mean_absolute_error(y_test, y_predicted)

print('MAE: %.3f' % mae)

MAE: 4979.608


### Выделение трех важных признаков. Метод выбора признаков на основе фильтров.

Выделим три наиболее важных признака с помощью метода K лучших пременных с использованием расчета статистики методом вычисления коэффициента корреляции Пирсона.

In [136]:
# создаем объект класса SelectKBest, передавая ему в качестве функции 
# метод определения коэффициента корреляции Присона и 
# требуемое количество важных признаков 
selector = SelectKBest(f_regression, k=3)
# применяем к созданному объекту SelectKBest метод fit передавая тренировочные выборку с таргетом
selector.fit(X_train, y_train)
# выводим полученные три важнейших признака 
selector.get_feature_names_out()

array(['year', 'condition', 'odometer'], dtype=object)

Создадим выборки с этими тремя важными признаками.

In [137]:
X_train_SKB = X_train.loc[:, ['year', 'condition', 'odometer']]
X_test_SKB = X_test.loc[:, ['year', 'condition', 'odometer']]

Создадим и обучим модель линейной регрессии на выбранных трех важных признаках. Сделаем предсказание на тестовой выборке.

In [138]:
# создаем обект класса модель линейной регрессии
model = LinearRegression()
# обучаем модель на тренировочных выборках
model.fit(X_train_SKB, y_train)
# делаем предсказание на тестовой выборке
y_predicted = model.predict(X_test_SKB)

Оценим резудьтат предсказания с помощью метрики MAE.

In [139]:
mae = mean_absolute_error(y_test, y_predicted)

print('MAE: %.3f' % mae)

MAE: 4660.119


### Выводы

Сравнение двух методов отбора признаков (выбора трех "важных" признаков) на тестовой выборке показывает преимущество метода K лучших пременных с использованием коэффициента корреляции Пирсона над методом рекурсивного исключения признаков. Значение MAE 4660 к 4979, соответственно.